In [ ]:
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text_gen = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    truncation=False,
    max_new_tokens=100,
    do_sample=True,
    return_full_text= True,
    temperature=0.7)

llm = HuggingFacePipeline(pipeline=text_gen)

prompt = PromptTemplate.from_template("아주 먼 옛날 {user_input} 이 살고 있었습니다. 그러던 어느날")
chain = prompt| llm
result1 = chain.invoke({"user_input": "토끼"})
print(result1)
len(result1)
result2 = chain.invoke({"user_input": "용"})
print(result2)
len(result2)


#1. 분류 노드
def classification(state):
    text = state.get("user_input", "")
    result = chain.invoke({"user_input": text}).strip()

    if (len(result) > 250):
        label = "long_answer"
    else: 
        label = "short_answer"
    return {**state, "label": label}

#2. 응답 노드
def long_answer(state):
    return {**state, "response": "장편소설"}
def short_answer(state):
    return {**state, "response": "단편 소설"}

def get_label(state):
    return state.get("label", "")

graph = StateGraph(dict)
graph.add_node("classification", classification)
graph.add_node("long_answer", long_answer)
graph.add_node("short_answer", short_answer)
graph.set_entry_point("classification")

graph.add_conditional_edges("classification", get_label, {
    "long_answer": "long_answer",
    "short_answer": "short_answer"
})

graph.add_edge("long_answer", END)
graph.add_edge("short_answer", END)

app = graph.compile()
user_input = input("동물 입력: ")
final_state = app.invoke({"user_input": user_input})
print("응답:", final_state.get("response", ""))


Device set to use cpu


아주 먼 옛날 토끼 이 살고 있었습니다. 그러던 어느날 토끼를 한 마리 잡아서 우리 집으로 데려다가 줬는데 그 토끼가 토끼가 아니라 토끼라고 해서 토끼가 아니었다는 사실 기억도 못합니다. 아~ 토끼가 아니라 토끼로 생각하시면 안 됩니다. 그~ 토끼가 토끼였죠. 아니 토끼가 아니니까 토끼라고 생각하시면 안 됩니다. 토끼도 아니니까 토끼입니다 토끼입니다 토끼입니다 토끼입니다 토끼입니다 토끼입니다 토끼입니다 토끼입니
아주 먼 옛날 용 이 살고 있었습니다. 그러던 어느날 님이 우리 할아버지와 그 집 아저씨한테서 와 와 와 와 와 와 와 와 와 와 와 와 와 를 데리고 나가셨습니다. 그때 님이 그 집 아주머니께 와 와 와 와 와 와 와 와 와 와 와 
응답: 장편소설


In [2]:
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


text_gen = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    truncation=False,
    max_new_tokens=100,
    do_sample=True,
    return_full_text= True,
    temperature=0.7)

llm = HuggingFacePipeline(pipeline=text_gen)
prompt = PromptTemplate.from_template("아주 먼 옛날 {user_input} 이 살고 있었습니다. 그러던 어느날")
chain = prompt| llm


#1. 분류 노드
def classification(state):
    text = state.get("user_infput", "")
    result = chain.invoke({"user_input": user_input}).strip()

    if (len(result) > 250):
        label = "long_answer"
    else: 
        label = "short_answer"
    return {**state, "label": label, "story": result}

#2. 응답 노드
def long_answer(state):
    res = "장편소설"
    print(res)
    return {**state, "response": res}
def short_answer(state):
    res = "단편 소설"
    print(res)
    return {**state, "response": res}

def ask_continue(state):
    reply = input("\n추가 질문하시겠습니까? (y/n): ").strip()
    if reply in ["예", "네", "y"]:
        state["continue"] = True
        qestion = input("추가 질문을 입력하세요: ")
        state["user_input"] = qestion
    else:
        state["continue"] = False
    return state

def shuld_continue(state):
    if state.get("continue"):
       answer = "replay"
    else:
       answer= "__end__"
    return answer

def replay_story(state):
    question= state.get("user_input", "")
    if "보여줘" in question :
        print("\n이전에 생성된 이야기 보여줘\n", state.get("story", "이야기가 없습니다."))
        return {**state, "response": "이야기 재생성 완료"}
    else:
        return classification(state)    


def get_label(state):
    return state.get("label", "")

graph = StateGraph(dict)
graph.add_node("classification", classification)
graph.add_node("long_answer", long_answer)
graph.add_node("short_answer", short_answer)
graph.add_node("ask_continue", ask_continue)
graph.add_node("replay_story", replay_story)

graph.set_entry_point("classification")

graph.add_conditional_edges("classification",get_label, {
                            "long_answer":"long_answer",
                            "short_answer":"short_answer"
})
graph.add_edge("long_answer", "ask_continue")
graph.add_edge("short_answer", "ask_continue")

graph.add_conditional_edges("ask_continue", shuld_continue, {
    "replay": "replay_story",
    "__end__": END
})

graph.add_edge("replay_story", "ask_continue")    

app = graph.compile()
user_input = input("동물 입력: ")
finanl_state = app.invoke({"user_input": user_input})
print("\n대화 종료")


Device set to use cpu


장편소설

이전에 생성된 이야기 보여줘
 아주 먼 옛날 토끼 이 살고 있었습니다. 그러던 어느날 토끼가 찾아왔습니다. 그때 토끼는 곧 그 친구에게 편지를 썼습니다. 그러자 친구가 토끼가 무슨 일을 하는지 물어보면서 토끼는 그 친구가 바로 그 친구를 위해 일한다는 것이 무슨 뜻인지 물어보았습니다. 그러자 토끼는 그 친구를 위해 일한다는 것이 무슨 뜻인지 물었습니다. 그러자 그 친구가 그 친구에게 편지를 써서 보냈습니다. 그 친구가 그 편지를 그 친구에게 보낸 바로 다음날이었습니다. 그 날 토끼는 그 친구에게 편지를 전해주었는데 그 친구는 그 친구를 위해 일을 한다는

대화 종료


In [26]:
#langgraph + lanchain + RAG

import pandas as pd
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.output_parsers import StrOutputParser

df = pd.read_csv('C:\\Users\\Leo\\code\\ch07_트랜스포머_RAG\\test03\\data\\data.csv', encoding='utf8')
df.head(8)

texts = df['text'].tolist()
docs = [Document(page_content=text) for text in texts]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chromaDB"
)

# LLM
model_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text_gen = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    truncation=False,
    max_new_tokens=100,
    do_sample=True,
    return_full_text= True,
    temperature=0.7)


llm = HuggingFacePipeline(pipeline=text_gen)
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

template = """ 다음 문맥을 사용하여 질문에 답변하세요.
답을 모르면 모른다고 말하세요.
간결하게 답변하세요

문맥: {context}
질문: {question}
답변:"""

prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {
        "context": retriever| format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

def ask_question(state):
    question = input("질문을 입력해주세요:")
    retieved_docs = retriever.invoke(question)
    answer = rag_chain.invoke(question)

    res = {
        "question": question,        
        "answer": answer,
        "source_docs": retieved_docs
    }
    return res

def get_answer(state):
    print("\n답변:", state["answer"])
    return state

def ask_reference(state):
    reply = input("\n참고 문서를 보겠습니까? (y/n): ").strip()
    return {**state, "see_reference":reply}

def get_reference(state):
    print("\n[참고 문서]:")
    for i, doc in enumerate(state["source_docs"], 1):
        print(f"{i}. {doc.page_content[:200]}...")
    return state

def ask_continue(state):
    reply = input("\n추가 질문하시겠습니까? (y/n): ").strip()
    return {**state, "continue": reply}
  
def reference_or_not(state):
    return "get_reference" if state.get("see_reference","") in ["예","네","y"] else "ask_continue"

def continue_or_not(state):
    return "ask_question" if state.get("continue","") in ["예","네","y"] else END

graph = StateGraph(dict)
graph.add_node("ask_question", ask_question)
graph.add_node("get_answer", get_answer)
graph.add_node("ask_reference", ask_reference)
graph.add_node("get_reference", get_reference)
graph.add_node("ask_continue", ask_continue)

graph.set_entry_point("ask_question")

graph.add_edge("ask_question", "get_answer")
graph.add_edge("get_answer", "ask_reference")
graph.add_conditional_edges("ask_reference", reference_or_not, {
    "get_reference": "get_reference",
    "ask_continue": "ask_continue"
})

graph.add_edge("get_reference", "ask_continue")
graph.add_conditional_edges("ask_continue", continue_or_not, {
    "ask_question": "ask_question",
    END: END
})

app = graph.compile()
app.invoke({})

Device set to use cpu



답변: Human:  다음 문맥을 사용하여 질문에 답변하세요.
답을 모르면 모른다고 말하세요.
간결하게 답변하세요

문맥: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.

MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.

MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.

MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES 공정 로그 보관 기간은?
답변: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
답변: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.
질문: MES 시스템

[참고 문서]:
1. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다....
2. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다....
3. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다....
4. MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다....


{'question': 'MES 공정 로그 보관 기간은?',
 'answer': 'Human:  다음 문맥을 사용하여 질문에 답변하세요.\n답을 모르면 모른다고 말하세요.\n간결하게 답변하세요\n\n문맥: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n\nMES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n\nMES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n\nMES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES 공정 로그 보관 기간은?\n답변: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n답변: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.\n질문: MES 시스템',
 'source_docs': [Document(id='95b91eb1-6210-4bae-b3f4-d5be1883c29c', metadata={}, page_content='MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.'),
  Document(id='c607a805-3af3-4491-9915-e6e338c003e2', metadata={}, page_content='MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.'),
  Document(id='e3268b9a-f08a-492b-9326-6dbaf4855fed', metadata={}, page_content='MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.'),
  Document(id='a267291c-1e50-4ac4-a92e-3f7967825f6a', metadata={}, page_content='MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.')],
 'see_ref

In [19]:

from langchain_chroma import Chroma
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")   
retriever = Chroma(
    persist_directory="./chromaDB",
    embedding_function=embedding_model
).as_retriever(search_kwargs={"k":3})

model_id = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer = AutoTokenizer.from_pretrained(model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_id)
text_gen = pipeline(
    "question-answering",
    model = qa_model,
    tokenizer = qa_tokenizer,
    device =-1,
    max_length=512,
    do_sample=False,
    temperature=0.1,
    trncation=True
)

def ask_question(state):
    question = input("질문을 입력해주세요:").strip()
    docs = retriever.invoke(question)
    top_docs = docs[:3]
    best_contexts = [doc.page_content for doc in top_docs]
    combined_context = "\n".join(best_contexts)
    result = text_gen(question=question, context=combined_context)
    

    res = {
        "question": question,        
        "answer": result['answer'],
        "source_docs": best_contexts
    }
    return res

def get_answer(state):
    print("\n질문:", state["question"])
    print("\n답변:", state["answer"])
    return state


def ask_reference(state):
    reply = input("\n참고 문서를 보겠습니까? (y/n): ").strip()
    state["show_reference"] = user_input.startswith(("예","네","y"))
    return state

def get_reference(state):
    if state.get("show_reference"):

        print("\n[참고 문서]\n:", state["context"])
    return state


def ask_continue(state):
    user_input = input("\n추가 질문하시겠습니까? (y/n): ").strip()
    state["continue"] = user_input.startswith("y")
    return state
  
def should_show_reference(state):
    return "get_reference" if state.get("show_reference") else "ask_continue"

def should_continue(state):
    return "ask_question" if state.get("continue")  else END



graph = StateGraph(dict)
graph.add_node("ask_question", ask_question)
graph.add_node("get_answer", get_answer)
graph.add_node("ask_reference", ask_reference)
graph.add_node("get_reference", get_reference)
graph.add_node("ask_continue", ask_continue)

graph.set_entry_point("ask_question")

graph.add_edge("ask_question", "get_answer")
graph.add_edge("get_answer", "ask_reference")
graph.add_conditional_edges("ask_reference", should_show_reference, {
    "get_reference": "get_reference",
    "ask_continue": "ask_continue"
})

graph.add_edge("get_reference", "ask_continue")
graph.add_conditional_edges("ask_continue", should_continue, {
    "ask_question": "ask_question",
    END: END
})

app = graph.compile()
app.invoke({})

Device set to use cpu



질문: MES 공정 로그 보관 기간은?

답변: 6개월간

질문: y

답변: 신입 엔지니어는 공정별 교육을


{'question': 'y',
 'answer': '신입 엔지니어는 공정별 교육을',
 'source_docs': ['신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다.',
  '신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다.',
  '신입 엔지니어는 공정별 교육을 모두 이수한 후 장비 조작이 가능합니다.'],
 'show_reference': False,
 'continue': False}

In [29]:
import uuid
from langchain_chroma import Chroma
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableConfig
from langchain_core.chat_history import BaseChatMessageHistory

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")   
retriever = Chroma(
    persist_directory="./chromaDB",
    embedding_function=embedding_model
).as_retriever(search_kwargs={"k":3})

model_id = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer = AutoTokenizer.from_pretrained(model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_id)
text_gen = pipeline(
    "question-answering",
    model = qa_model,
    tokenizer = qa_tokenizer,
    device =-1,
    max_length=512,
    do_sample=False,
    temperature=0.1,
    trncation=True
)
llm = HuggingFacePipeline(pipeline=text_gen)
chats_by_ssetion_id = {}

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chats_by_ssetion_id:
        chats_by_ssetion_id[session_id] = InMemoryChatMessageHistory()
    return chats_by_ssetion_id[session_id]

def ask_question(state, config:RunnableConfig):
    session_id = config["configurable"]["session_id"]
    chat_history = get_chat_history(session_id)

    question = input("질문을 입력해주세요:").strip()
    docs = retriever.invoke(question)
    top_docs = docs[:3]
    best_contexts = [doc.page_content for doc in top_docs]
    combined_context = "\n".join(best_contexts)

    history_text = get_buffer_string(chat_history.messages)
    full_context = f"{history_text}\n\n{combined_context}" if history_text else combined_context

    result = text_gen(question=question, context=full_context)
    chat_history.add_user_message(question)
    chat_history.add_ai_message(result['answer'])
    return {
        "question": question,        
        "answer": result['answer'],
        "context": best_contexts
    }


def get_answer(state):
    print(f"\n질문:", state['question'])
    print(f"\n답변:", state['answer'])
    return state


def ask_reference(state):
    reply = input("\n참고 문서를 보겠습니까? (y/n): ").strip()
    state["show_reference"] = user_input.startswith("y")
    return state

def get_reference(state):
    if state.get("show_reference"):
        print ("\n참고문서:\n")
        for i, ctx in enumerate(state["context"], 1):
            print(f"[{i}] {ctx}\n")
    return state


def ask_continue(state):
    user_input = input("\n추가 질문하시겠습니까? (y/n): ").strip()
    state["continue"] = user_input.startswith("y")
    return state
  
def should_show_reference(state):
    return "get_reference" if state.get("show_reference") else "ask_continue"

def should_continue(state):
    return "ask_question" if state.get("continue")  else END


graph = StateGraph(dict)
graph.add_node("ask_question", ask_question)
graph.add_node("get_answer", get_answer)
graph.add_node("ask_reference", ask_reference)
graph.add_node("get_reference", get_reference)
graph.add_node("ask_continue", ask_continue)

graph.set_entry_point("ask_question")

graph.add_edge("ask_question", "get_answer")
graph.add_edge("get_answer", "ask_reference")
graph.add_conditional_edges("ask_reference", should_show_reference, {
    "get_reference": "get_reference",
    "ask_continue": "ask_continue"
})

graph.add_edge("get_reference", "ask_continue")
graph.add_conditional_edges("ask_continue", should_continue, {
    "ask_question": "ask_question",
    END: END
})

session_id = str(uuid.uuid4())
config = {"configurable": {"session_id": session_id}}
app = graph.compile()
app.invoke({}, config= config)

Device set to use cpu



질문: MES 공정 로그 보관 기간은?

답변: 6개월간


{'question': 'MES 공정 로그 보관 기간은?',
 'answer': '6개월간',
 'context': ['MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.',
  'MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.',
  'MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.'],
 'show_reference': False,
 'continue': False}

In [ ]:
# Word 파일을 읽어서 ChromaDB에 저장하는 예시 함수
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def save_word_to_chromadb(docx_path, chroma_dir, embedding_model_name):
    # 1. Word 파일 읽기
    loader = UnstructuredWordDocumentLoader(docx_path)
    docs = loader.load()
    
    # 2. 임베딩 모델 준비
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    
    # 3. ChromaDB에 저장
    vectorstore = Chroma.from_documents(
    	documents=docs,
    	embedding=embedding_model,
    	persist_directory=chroma_dir
    )
    print(f"{len(docs)}개의 문서가 ChromaDB에 저장되었습니다.")
    return vectorstore

# 사용 예시
chroma_dir = "./chromaDB"
docx_path = "NewBie_개발환경가이드1.docx"
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
vectorstore = save_word_to_chromadb(docx_path, chroma_dir, embedding_model_name)